<a href="https://colab.research.google.com/github/KIQVinicius/Data-Science/blob/master/AnomaliasJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
import numpy as np
from datetime import date

In [3]:
dataset = pd.read_csv("MIPS1.csv")

In [4]:
porcentagem = float(dataset['Porcentagem'][1])/100 + 1
dataset.columns = ['Index', 'JOBNAME', 'DATE', 'Valor_MIPS', 'Porcentagem', 'Chave']

In [5]:
dataset = dataset.drop(columns=["Porcentagem", "Chave"], axis=1)

In [6]:
dataset['DATE'] = pd.to_datetime(dataset['DATE'])

In [7]:
first_day = dataset['DATE'].min()
last_day = dataset['DATE'].max()

In [8]:
first_day

Timestamp('2019-01-11 00:00:00')

In [9]:
last_day

Timestamp('2020-12-05 00:00:00')

In [10]:
dataset_first_day = dataset.query("DATE == " +"'" + str(first_day) + "'")
dataset_last_day = dataset.query("DATE == " +"'" + str(last_day) + "'")

In [11]:
dt = last_day.date() - first_day.date()

In [12]:
weeks = dt.days/30

In [13]:
weeks = int(weeks)

In [14]:
weeks

23

In [15]:
data = first_day

In [16]:
data+ timedelta(days = 30)

Timestamp('2019-02-10 00:00:00')

In [17]:
frequencia = pd.DataFrame(dataset['JOBNAME'].value_counts())
frequencia.reset_index(inplace=True)
frequencia.columns = ['JOBNAME', 'Frequencia']

In [18]:
frequencia = frequencia.query("Frequencia > 1")

In [19]:
dataset = dataset[dataset['JOBNAME'].isin(frequencia['JOBNAME'])]
dataset.head()

,Index,JOBNAME,DATE,Valor_MIPS
0,0,FG94BM07,2020-05-24,0.08
1,1,FG94BM09,2020-05-24,0.03
2,2,FG94BM19,2020-05-24,0.02
3,3,FG94BM20,2020-05-24,0.48
4,4,FG94BM21,2020-05-24,0.02


In [20]:
pd.DataFrame(dataset['JOBNAME'].value_counts())
dataset = dataset.drop(columns=["Index"], axis=1)

In [21]:
data = first_day
tabela_final = dataset.iloc[0:0]
soma = 0
dataset_semana_anterior = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = 30)) + "' ")
data = data+ timedelta(days = 30)
dataset_semana_anterior = dataset_semana_anterior[dataset_semana_anterior['Valor_MIPS']>0].groupby('JOBNAME')
dataset_semana_anterior = pd.DataFrame(dataset_semana_anterior['Valor_MIPS'].mean())
dataset_semana_anterior.reset_index(inplace =True)
data = data+ timedelta(days = 30)
for week in range(0, weeks, 1):
  print("------------------------")
  dataset_semana = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = 30)) + "' ")
  dataset = dataset.query("DATE >= " + " '"+ str(data+ timedelta(days = 30))+ "' ")
  print("Dataset: " + str(dataset.shape[0]))
  print("Final: " + str(tabela_final.shape[0]))
  soma = soma + dataset_semana.shape[0]
  #dataset_semana = dataset_semana[dataset_semana['Valor_MIPS']>0].groupby('JOBNAME')
  #dataset_semana = pd.DataFrame(dataset_semana['Valor_MIPS'].mean())
  print(tabela_final)
  for index, row in dataset_semana_anterior.iterrows():
    tabela_final = tabela_final.append(dataset_semana.query("JOBNAME == "+"'"+row['JOBNAME']+"'" + " and Valor_MIPS > "+ str(float(row['Valor_MIPS']) * porcentagem)))
  
  dataset_semana_anterior = dataset_semana[dataset_semana['Valor_MIPS']>0].groupby('JOBNAME')
  dataset_semana_anterior = pd.DataFrame(dataset_semana_anterior['Valor_MIPS'].mean())
  dataset_semana_anterior.reset_index(inplace=True)
  data = data+ timedelta(days = 30)

------------------------
Dataset: 1088926
Final: 0
Empty DataFrame
Columns: [JOBNAME, DATE, Valor_MIPS]
Index: []
------------------------
Dataset: 1074959
Final: 3708
         JOBNAME       DATE  Valor_MIPS
92922     AATTSO 2019-03-12        0.19
355585    AATTSO 2019-04-10        0.26
92923   ANTAS000 2019-03-12        0.32
92924    ANTMAIN 2019-03-12        0.14
92925       AOPD 2019-03-12        2.58
...          ...        ...         ...
98132   WEKACHIN 2019-03-12        1.10
360602  WEKACHIN 2019-04-10        1.01
98133   WEKAMSTR 2019-03-12        2.77
98134        WLM 2019-03-12      133.03
98135      XCFAS 2019-03-12       18.17

[3708 rows x 3 columns]
------------------------
Dataset: 1064519
Final: 7370
         JOBNAME       DATE  Valor_MIPS
92922     AATTSO 2019-03-12        0.19
355585    AATTSO 2019-04-10        0.26
92923   ANTAS000 2019-03-12        0.32
92924    ANTMAIN 2019-03-12        0.14
92925       AOPD 2019-03-12        2.58
...          ...        ...      

UndefinedVariableError: ignored

In [ ]:
tabela_final['DATE'].value_counts()